In [1]:
using Flux

In [7]:
f(x) = 3x^2 + 2x + 1;

In [10]:
df(x) = Tracker.gradient(f, x; nest = true)[1];

UndefVarError: UndefVarError: x not defined

In [15]:
df(2)

14.0 (tracked)

---

In [1]:
using Flux

In [17]:
 W = param(2)

2.0 (tracked)

In [18]:
b = param(3)

3.0 (tracked)

In [19]:
f(x) = W * x + b;

In [20]:
f(1)

5.0 (tracked)

In [23]:
grads = Tracker.gradient(() -> f(4), params(W, b))

Grads(...)


In [24]:
typeof(grads)

Tracker.Grads

In [25]:
grads[W]

4.0 (tracked)

In [26]:
grads[b]

1.0 (tracked)

---

In [1]:
using BenchmarkTools

In [2]:
using SparseArrays
import SparseArrays.sparse

In [27]:
rand()

0.08430148944781934

In [2]:
struct SparseCOO
    row_indices
    column_indices
    values
    row_count
    column_count
end

In [12]:
my_sparse = SparseCOO([1,2,3], [1,2,3], [1, 2, 3], 3, 3)

SparseCOO([1, 2, 3], [1, 2, 3], [1, 2, 3], 3, 3)

In [13]:
sparse(m::SparseCOO) = sparse(m.row_indices, m.column_indices, m.values, m.row_count, m.column_count)

sparse (generic function with 24 methods)

In [15]:
typeof(sparse(my_sparse))

SparseMatrixCSC{Int64,Int64}

In [31]:
A = sparse([2,5,3,8,6], [2,4,5,7,9], 1, 10, 10)

10×10 SparseMatrixCSC{Int64,Int64} with 5 stored entries:
  [2 ,  2]  =  1
  [5 ,  4]  =  1
  [3 ,  5]  =  1
  [8 ,  7]  =  1
  [6 ,  9]  =  1

In [34]:
A.rowval

5-element Array{Int64,1}:
 2
 5
 3
 8
 6

In [32]:
A.colptr

11-element Array{Int64,1}:
 1
 1
 2
 2
 3
 4
 4
 5
 5
 6
 6

In [33]:
Matrix(A)

10×10 Array{Int64,2}:
 0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  1  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0

In [31]:
A

3×3 SparseMatrixCSC{Int64,Int64} with 1 stored entry:
  [2, 2]  =  2

In [32]:
R = sparsevec([2], [5])

2-element SparseVector{Int64,Int64} with 1 stored entry:
  [2]  =  5

In [17]:
M = sprand(100, 1000, 0.1);
N = sprand(10, 100, 0.1);

In [16]:
function mutate(M, row_indices, column_indices, values)
    I, J, V = findnz(M)
    append!(I, row_indices)
    append!(J, column_indices)
    append!(V, values)
    return sparse(I, J, V)
end

function mutate_(M, row_indices, column_indices, values)
    for i ∈ I, j ∈ J, val ∈ values
        M[i, j] = val
    end
    return M
end

mutate_ (generic function with 1 method)

In [19]:
I, J, V = findnz(N);

In [20]:
@benchmark mutate($M, $I, $J, $V)

BenchmarkTools.Trial: 
  memory estimate:  1.06 MiB
  allocs estimate:  22
  --------------
  minimum time:     406.653 μs (0.00% GC)
  median time:      472.603 μs (0.00% GC)
  mean time:        528.496 μs (7.53% GC)
  maximum time:     50.251 ms (99.02% GC)
  --------------
  samples:          9395
  evals/sample:     1

In [21]:
@benchmark mutate_($M, $I, $J, $V)

InterruptException: InterruptException:

In [37]:
Matrix(sprand(3, 4, 0.2))

3×4 Array{Float64,2}:
 0.0  0.0       0.0  0.38615 
 0.0  0.586285  0.0  0.980001
 0.0  0.626113  0.0  0.0     

In [55]:
M = sprand(300, 400, 0.5);
w = sprand(400, 0.5);

In [56]:
Matrix(M)

300×400 Array{Float64,2}:
 0.0811027  0.0          0.462146   …  0.852927  0.677511   0.83239  
 0.572516   0.0          0.166363      0.0       0.0        0.0      
 0.0        0.0          0.0           0.889867  0.0        0.985618 
 0.223376   0.932677     0.0           0.0       0.0        0.0      
 0.0        0.0          0.722149      0.0       0.635984   0.0      
 0.490164   0.960898     0.0        …  0.282651  0.183361   0.13568  
 0.0        0.178784     0.734072      0.0       0.0        0.18761  
 0.0879198  0.827821     0.0           0.0       0.0        0.0      
 0.409745   0.0217611    0.879398      0.285096  0.713446   0.0      
 0.743483   0.0          0.0           0.0       0.0        0.577751 
 0.0        0.398693     0.0        …  0.58215   0.0425201  0.452731 
 0.0        0.0          0.258192      0.809208  0.0        0.0      
 0.136027   0.0          0.928268      0.0       0.0        0.919195 
 ⋮                                  ⋱                           

In [57]:
M.colptr

401-element Array{Int64,1}:
     1
   159
   312
   461
   612
   768
   924
  1078
  1248
  1405
  1549
  1692
  1833
     ⋮
 58378
 58522
 58665
 58805
 58949
 59117
 59263
 59427
 59572
 59707
 59847
 59997

In [58]:
Vector(w)

400-element Array{Float64,1}:
 0.0                
 0.5915055934162623 
 0.0                
 0.0                
 0.0                
 0.0                
 0.6832170270236613 
 0.0                
 0.0                
 0.5852423760096379 
 0.0                
 0.9186724642695827 
 0.0                
 ⋮                  
 0.09332612424092801
 0.0                
 0.7594118742966263 
 0.22129865386312053
 0.0                
 0.656353706395842  
 0.0                
 0.5786432248148419 
 0.5572922885314855 
 0.0                
 0.0                
 0.2971123701091638 

In [59]:
w.nzind

186-element Array{Int64,1}:
   2
   7
  10
  12
  14
  15
  16
  17
  18
  19
  20
  21
  25
   ⋮
 373
 376
 380
 383
 387
 389
 391
 392
 394
 396
 397
 400

In [50]:
Matrix(M) * Vector(w)

3-element Array{Float64,1}:
 0.0                 
 0.27462334281017947 
 0.004576686054722911

In [49]:
Vector(M * w)

3-element Array{Float64,1}:
 0.0                 
 0.27462334281017947 
 0.004576686054722911

---

In [2]:
using Flux
using SparseArrays

In [3]:
W = sprand(2, 3, 0.5)

2×3 SparseMatrixCSC{Float64,Int64} with 5 stored entries:
  [1, 1]  =  0.818541
  [2, 1]  =  0.892875
  [1, 2]  =  0.71225
  [2, 2]  =  0.876172
  [1, 3]  =  0.520788

In [4]:
Matrix(W)

2×3 Array{Float64,2}:
 0.818541  0.71225   0.520788
 0.892875  0.876172  0.0     

In [5]:
b = sprand(2, 0.5);

In [6]:
b

2-element SparseVector{Float64,Int64} with 1 stored entry:
  [1]  =  0.100559

In [7]:
f(x) = sum(W * x + b);

In [8]:
W = param(W);
b = param(b);

In [9]:
df(x) = Tracker.gradient(() -> f(x), params(W, b));

In [10]:
x_0 = 2 * ones(3);

In [11]:
grads = df(x_0);

In [13]:
grads[b]

Tracked 2-element SparseVector{Float64,Int64}:
 1.0
 1.0

---

In [2]:
using Pkg; Pkg.status("Flux")

    Status `~/.julia/environments/v1.3/Project.toml`
  [587475ba] Flux v0.10.0


In [65]:
using Flux
using Flux.Data.MNIST
using Flux: onehotbatch, Chain, Dense, train!, crossentropy, mse, Params, gradient
using Base.Iterators: repeated

In [2]:
using Test

In [5]:
using Flux
using Flux: Chain, crossentropy, gradient
using SparseArrays
struct SprAffine{F,S,T}
    W::S
    b::T
    σ::F
end
SprAffine(in::Integer, out::Integer, σ::Function) = SprAffine(sprandn(out, in, 0.5), sprandn(out, 0.5), σ)
(m::SprAffine)(x) = m.σ.(m.W * x .+ m.b)
Flux.@functor SprAffine
loss(x, y) = crossentropy(m(x), y)
m = Chain(SprAffine(4, 3, σ), SprAffine(3, 2, σ), softmax);

In [8]:
typeof(loss(sparse(1:4), sparse(1:2)))

Float64

In [7]:
gradient(params(m)) do
    loss(sparse(1:4), sparse(1:2))
end

MethodError: MethodError: no method matching zero(::Type{Tuple{Float64,Zygote.var"#916#back#380"{Zygote.var"#378#379"{Float64}}}})
Closest candidates are:
  zero(!Matched::Type{Missing}) at missing.jl:103
  zero(!Matched::Type{LibGit2.GitHash}) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/LibGit2/src/oid.jl:220
  zero(!Matched::Type{Pkg.Resolve.VersionWeights.VersionWeight}) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Pkg/src/resolve/VersionWeights.jl:19
  ...

In [35]:
using SparseArrays

In [36]:
struct SprAffine{S,T}
    W::S
    b::T
end

In [54]:
SprAffine(in::Integer, out::Integer) = SprAffine(sprandn(out, in, 0.5), sprandn(out, 0.5))
(m::SprAffine)(x) =m.W * x .+ m.b

In [39]:
Flux.@functor SprAffine

In [13]:
struct Affine{S,T}
    W::S
    b::T
end

In [14]:
Affine(in::Integer, out::Integer) = Affine(randn(out, in), randn(out))
(m::Affine)(x) = m.W * x .+ m.b

In [15]:
Flux.@functor Affine

In [16]:
function fizzbuzz(x::Int)
    is_divisible_by_three = x % 3 == 0
    is_divisible_by_five = x % 5 == 0

    if is_divisible_by_three & is_divisible_by_five
        return "fizzbuzz"
    elseif is_divisible_by_three
        return "fizz"
    elseif is_divisible_by_five
        return "buzz"
    else
        return "else"
    end
end

fizzbuzz (generic function with 1 method)

In [17]:
const LABELS = ["fizz", "buzz", "fizzbuzz", "else"];

In [18]:
@test fizzbuzz.([3, 5, 15, 98]) == LABELS

Test Passed

In [40]:
raw_x = 1:100;
raw_y = fizzbuzz.(raw_x);

In [41]:
features(x) = float.([x % 3, x % 5, x % 15])
features(x::AbstractArray) = hcat(features.(x)...)

features (generic function with 2 methods)

In [21]:
X = features(raw_x);
y = onehotbatch(raw_y, LABELS);

In [22]:
#m = Chain(Dense(3, 10), Dense(10, 4), softmax)

In [75]:
m = Chain(SprAffine(10, 5), softmax);

In [76]:
# m =
# Chain(
#     Affine(3, 10),
#     Affine(10, 4),
#     softmax
# )

In [77]:
loss(x, y) = crossentropy(m(x), y)

loss (generic function with 1 method)

In [48]:
opt = ADAM()

ADAM(0.001, (0.9, 0.999), IdDict{Any,Any}())

In [49]:
deepbuzz(x) = (a = argmax(m(features(x))); a == 4 ? x : LABELS[a])

deepbuzz (generic function with 1 method)

In [50]:
function monitor(e)
    print("epoch $(lpad(e, 4)): loss = $(round(loss(X,y); digits=4)) | ")
    @show deepbuzz.([3, 5, 15, 98])
end

monitor (generic function with 1 method)

In [51]:
@which train!

Flux.Optimise

In [79]:
m(sparse(1:10))

5-element SparseVector{Float64,Int64} with 5 stored entries:
  [1]  =  1.86481e-15
  [2]  =  1.59814e-8
  [3]  =  0.0211817
  [4]  =  0.978813
  [5]  =  5.52872e-6

In [9]:
loss(sparse(1:10), sparse(1:5))

DimensionMismatch: DimensionMismatch("")

In [81]:
gradient(params(m)) do
    loss(sparse(1:10), sparse(1:5))
end

MethodError: MethodError: no method matching zero(::Type{Tuple{Float64,Zygote.var"#916#back#380"{Zygote.var"#378#379"{Float64}}}})
Closest candidates are:
  zero(!Matched::Type{Missing}) at missing.jl:103
  zero(!Matched::Type{LibGit2.GitHash}) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/LibGit2/src/oid.jl:220
  zero(!Matched::Type{Pkg.Resolve.VersionWeights.VersionWeight}) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Pkg/src/resolve/VersionWeights.jl:19
  ...

In [60]:
train!(loss, params(m), [(X, y)], opt)

MethodError: MethodError: no method matching zero(::Type{Tuple{Float64,Zygote.var"#916#back#380"{Zygote.var"#378#379"{Float64}}}})
Closest candidates are:
  zero(!Matched::Type{Missing}) at missing.jl:103
  zero(!Matched::Type{LibGit2.GitHash}) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/LibGit2/src/oid.jl:220
  zero(!Matched::Type{Pkg.Resolve.VersionWeights.VersionWeight}) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Pkg/src/resolve/VersionWeights.jl:19
  ...

In [34]:
# Training
for e in 0:1000
    train!(loss, params(m), [(X, y)], opt)
    if e % 50 == 0; monitor(e) end
end

epoch    0: loss = 15.3308 | deepbuzz.([3, 5, 15, 98]) = ["fizzbuzz", "buzz", "buzz", "fizzbuzz"]
epoch   50: loss = 6.956 | deepbuzz.([3, 5, 15, 98]) = ["fizzbuzz", "buzz", "buzz", "fizz"]
epoch  100: loss = 4.8983 | deepbuzz.([3, 5, 15, 98]) = ["fizz", "buzz", "buzz", "fizz"]
epoch  150: loss = 3.8004 | deepbuzz.([3, 5, 15, 98]) = ["fizz", "buzz", "buzz", "fizz"]
epoch  200: loss = 2.8627 | deepbuzz.([3, 5, 15, 98]) = ["fizz", "buzz", "buzz", "fizz"]
epoch  250: loss = 2.0293 | deepbuzz.([3, 5, 15, 98]) = ["fizz", "buzz", "buzz", "fizz"]
epoch  300: loss = 1.3288 | deepbuzz.([3, 5, 15, 98]) = Any["fizz", "buzz", "buzz", 98]
epoch  350: loss = 0.8912 | deepbuzz.([3, 5, 15, 98]) = Any["fizz", "buzz", "buzz", 98]
epoch  400: loss = 0.6748 | deepbuzz.([3, 5, 15, 98]) = Any[3, "buzz", "buzz", 98]
epoch  450: loss = 0.5544 | deepbuzz.([3, 5, 15, 98]) = Any[3, "buzz", "buzz", 98]
epoch  500: loss = 0.4704 | deepbuzz.([3, 5, 15, 98]) = Any[3, "buzz", "buzz", 98]
epoch  550: loss = 0.4042 | d